In [2]:
# import dependencies
import os
import csv
from pandas import DataFrame
import pandas as pd
import numpy as np
from datetime import datetime
import datetime as dt
import glob

In [3]:
# display column names in a csv  
path = '../data_collection_munging/parking_citations/data/citations_db_ready/citations_0000000001.csv'
headers_list = pd.read_csv(path, sep=",")
headers_list.head(2)

/Users/pavelaltukhov/anaconda3/envs/meters_ml_3.6/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3049: DtypeWarning: Columns (11) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,agency,body_style,color,fine_amount,issue_date,latitude,location,longitude,make,meter_id,route,ticket_number,violation_code,violation_description
0,2,PA,BK,58.0,2015-12-21,33.945212,200 WORLD WAY,-118.401151,NISS,NaN,2R2,1104803000,8939,WHITE CURB
1,2,PA,WH,NaN,2015-12-26,33.945212,100 WORLD WAY,-118.401009,ACUR,NaN,2F11,1104820732,000,17104h


# Graph  1  |    Citations per Year / Month

In [40]:
# loop over csv files and extract columns needed

path = '../data_collection_munging/parking_citations/data/citations_db_ready/'
all_files = glob.glob(path + "/*.csv")

li = []

for filename in all_files:
    df = pd.read_csv(filename, sep=",", usecols=[4, 11])
    li.append(df)
new_df = pd.concat(li)
print(f'There are: {len(new_df)} records in dataframe')
new_df.head()

/Users/pavelaltukhov/anaconda3/envs/meters_ml_3.6/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3049: DtypeWarning: Columns (11) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


There are: 8076329 records in dataframe


,issue_date,ticket_number
0,2015-12-21,1104803000
1,2015-12-26,1104820732
2,2015-12-21,1108321723
3,2015-12-19,1109139006
4,2015-12-26,1109455266


In [41]:
# convert "issue_date" column to a date 
new_df['issue_date'] = pd.to_datetime(new_df['issue_date'], dayfirst=True)

In [43]:
# sort dataframe rows between two dates-----(2014 - 2019)
start_date = "2014-01-01"
end_date = "2019-12-31"

mask = (new_df['issue_date'] > start_date) & (new_df['issue_date'] <= end_date)
new_df.loc[mask]
citations_2014_2019 = new_df.loc[mask]

print(f'There are: {len(citations_2014_2019)} records as of {end_date}')
print("=============================================================")
print(f'{len(new_df)-len(citations_2014_2019)} records were disregarded')
citations_2014_2019.head()

There are: 8076220 records as of 2019-12-31
109 records were disregarded


,issue_date,ticket_number
0,2015-12-21,1104803000
1,2015-12-26,1104820732
2,2015-12-21,1108321723
3,2015-12-19,1109139006
4,2015-12-26,1109455266


In [44]:
# put year, month, and a day into separate columns
citations_2014_2019['Year'] = citations_2014_2019['issue_date'].dt.year
citations_2014_2019['Month'] = citations_2014_2019['issue_date'].dt.month
# citations_2015_2018['Day'] = citations_2015_2018['issue_date'].dt.day
citations_2014_2019.drop(["issue_date"], axis = 1, inplace = True)

citations_2014_2019.head()

/Users/pavelaltukhov/anaconda3/envs/meters_ml_3.6/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/Users/pavelaltukhov/anaconda3/envs/meters_ml_3.6/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/Users/pavelaltukhov/anaconda3/envs/meters_ml_3.6/lib/python3.6/site-packages/pandas/core/frame.py:3940: SettingWithCopyWarning: 
A value is trying to be set on a copy

,ticket_number,Year,Month
0,1104803000,2015,12
1,1104820732,2015,12
2,1108321723,2015,12
3,1109139006,2015,12
4,1109455266,2015,12


In [46]:
# count citations by year and month
df_top_frequency = citations_2014_2019.groupby(['Year'])['ticket_number'].agg(
    {"counts": len}).sort_values("Year", ascending=True).reset_index()

df_top_frequency.head()

/Users/pavelaltukhov/anaconda3/envs/meters_ml_3.6/lib/python3.6/site-packages/ipykernel_launcher.py:3: FutureWarning: using a dict on a Series for aggregation
is deprecated and will be removed in a future version
  This is separate from the ipykernel package so we can avoid doing imports until


,Year,counts
0,2014,34207
1,2015,1542798
2,2016,1864010
3,2017,2154788
4,2018,1772390


In [47]:
df_top_frequency.to_csv("visualization_data/graph_1_citations_by_yyyy.csv", encoding = 'utf-8', index = False)

# Map Visualization 1 | Areas with Highest Number of Tickets

In [126]:
# loop over csv files and extract columns needed

path = '../data_collection_munging/parking_citations/data/citations_db_ready/'
all_files = glob.glob(path + "/*.csv")

li = []

for filename in all_files:
    df = pd.read_csv(filename, sep=",", usecols=[5, 6, 7, 11])
    li.append(df)
new_df = pd.concat(li)
print(f'There are: {len(new_df)} records in dataframe')
new_df.head()

/Users/pavelaltukhov/anaconda3/envs/meters_ml_3.6/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3049: DtypeWarning: Columns (11) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


There are: 8076329 records in dataframe


,latitude,location,longitude,ticket_number
0,33.945212,200 WORLD WAY,-118.401151,1104803000
1,33.945212,100 WORLD WAY,-118.401009,1104820732
2,34.185788,10635 FRIAR ST,-118.363082,1108321723
3,34.157607,14006 RIVERSIDE DR,-118.438092,1109139006
4,34.126932,3200 CANYON DR,-118.311225,1109455266


In [129]:
# count citations by year and month
df_top_frequency = new_df.groupby(['location', 'latitude', 'longitude'])['ticket_number'].agg(
    {"counts": len}).sort_values("counts", ascending=False).reset_index()

df_top_frequency

/Users/pavelaltukhov/anaconda3/envs/meters_ml_3.6/lib/python3.6/site-packages/ipykernel_launcher.py:3: FutureWarning: using a dict on a Series for aggregation
is deprecated and will be removed in a future version
  This is separate from the ipykernel package so we can avoid doing imports until


,location,latitude,longitude,counts
0,1301 ELECTRIC AVE,33.991691,-118.467910,8330
1,1600 IRVING TABOR CT,33.990358,-118.463691,6318
2,101 LARCHMONT BL N,34.073023,-118.323530,4591
3,2110 CORINTH AVE,34.038852,-118.443275,3910
4,4301 TUJUNGA AV,34.147865,-118.378977,3379
5,2377 MIDVALE AVE,34.040144,-118.429894,3347
6,7000 HAWTHORN AVE,34.100334,-118.341399,3337
7,11100 WEDDINGTON ST,34.166995,-118.372456,3295
8,1152 LEMOYNE ST,34.076191,-118.260221,3250
9,3101 GLENDALE BL,34.116139,-118.263042,3199


In [132]:
df_top_frequency_1 = df_top_frequency[df_top_frequency.counts > 50]
df_top_frequency_1.head()

,location,latitude,longitude,counts
0,1301 ELECTRIC AVE,33.991691,-118.467910,8330
1,1600 IRVING TABOR CT,33.990358,-118.463691,6318
2,101 LARCHMONT BL N,34.073023,-118.323530,4591
3,2110 CORINTH AVE,34.038852,-118.443275,3910
4,4301 TUJUNGA AV,34.147865,-118.378977,3379


In [133]:
df_top_frequency_1.to_csv("visualization_data/map_1_citations_areas.csv", encoding = 'utf-8', index = False)

# Graph  2  |    Top 10 Parking Violations

In [48]:
# loop over csv files and extract columns needed

path = '../data_collection_munging/parking_citations/data/citations_db_ready/'
all_files = glob.glob(path + "/*.csv")

li = []

for filename in all_files:
    df = pd.read_csv(filename, sep=",", usecols=[4, 11, 13])
    li.append(df)
new_df = pd.concat(li)
print(f'There are: {len(new_df)} records in dataframe')
new_df.head()

/Users/pavelaltukhov/anaconda3/envs/meters_ml_3.6/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3049: DtypeWarning: Columns (11) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


There are: 8076329 records in dataframe


,issue_date,ticket_number,violation_description
0,2015-12-21,1104803000,WHITE CURB
1,2015-12-26,1104820732,17104h
2,2015-12-21,1108321723,RED ZONE
3,2015-12-19,1109139006,HANDICAP/NO DP ID
4,2015-12-26,1109455266,PK IN PROH AREA


In [49]:
# convert "issue_date" column to a date 
new_df['issue_date'] = pd.to_datetime(new_df['issue_date'], dayfirst=True)

In [50]:
# sort dataframe rows between two dates-----(2014 - 2019)
start_date = "2014-01-01"
end_date = "2019-12-31"

mask = (new_df['issue_date'] > start_date) & (new_df['issue_date'] <= end_date)
new_df.loc[mask]
violations_2014_2019 = new_df.loc[mask]

print(f'There are: {len(violations_2014_2019)} records as of {end_date}')
print("=============================================================")
print(f'{len(new_df)-len(violations_2014_2019)} records were disregarded')
violations_2014_2019.head()

There are: 8076220 records as of 2019-12-31
109 records were disregarded


,issue_date,ticket_number,violation_description
0,2015-12-21,1104803000,WHITE CURB
1,2015-12-26,1104820732,17104h
2,2015-12-21,1108321723,RED ZONE
3,2015-12-19,1109139006,HANDICAP/NO DP ID
4,2015-12-26,1109455266,PK IN PROH AREA


In [51]:
# put year, month, and a day into separate columns
violations_2014_2019['Year'] = violations_2014_2019['issue_date'].dt.year
violations_2014_2019.drop(["issue_date"], axis = 1, inplace = True)

violations_2014_2019.head()

/Users/pavelaltukhov/anaconda3/envs/meters_ml_3.6/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/Users/pavelaltukhov/anaconda3/envs/meters_ml_3.6/lib/python3.6/site-packages/pandas/core/frame.py:3940: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


,ticket_number,violation_description,Year
0,1104803000,WHITE CURB,2015
1,1104820732,17104h,2015
2,1108321723,RED ZONE,2015
3,1109139006,HANDICAP/NO DP ID,2015
4,1109455266,PK IN PROH AREA,2015


In [53]:
# count citations by year and month
df_top_frequency = violations_2014_2019.groupby(['violation_description'])['ticket_number'].agg(
    {"counts": len}).sort_values("counts", ascending=False).reset_index()

df_top_frequency

/Users/pavelaltukhov/anaconda3/envs/meters_ml_3.6/lib/python3.6/site-packages/ipykernel_launcher.py:3: FutureWarning: using a dict on a Series for aggregation
is deprecated and will be removed in a future version
  This is separate from the ipykernel package so we can avoid doing imports until


,violation_description,counts
0,NO PARK/STREET CLEAN,2253925
1,METER EXP.,1567254
2,PREFERENTIAL PARKING,605718
3,RED ZONE,598872
4,DISPLAY OF TABS,469719
5,NO PARKING,364420
6,DISPLAY OF PLATES,232441
7,WHITE ZONE,193300
8,PARKED OVER TIME LIMIT,167679
9,NO STOP/STANDING,149860


In [54]:
df_top_frequency.to_csv("visualization_data/graph_2_top10_violations.csv", encoding = 'utf-8', index = False)

# Graph  3  |    Parking Tickets Revenue

In [87]:
# loop over csv files and extract columns needed

path = '../data_collection_munging/parking_citations/data/citations_db_ready/'
all_files = glob.glob(path + "/*.csv")

li = []

for filename in all_files:
    df = pd.read_csv(filename, sep=",", usecols=[3, 4, 5, 7, 11, 13])
    li.append(df)
new_df = pd.concat(li)
print(f'There are: {len(new_df)} records in dataframe')
new_df.head()

/Users/pavelaltukhov/anaconda3/envs/meters_ml_3.6/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3049: DtypeWarning: Columns (11) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


There are: 8076329 records in dataframe


,fine_amount,issue_date,latitude,longitude,ticket_number,violation_description
0,58.0,2015-12-21,33.945212,-118.401151,1104803000,WHITE CURB
1,NaN,2015-12-26,33.945212,-118.401009,1104820732,17104h
2,93.0,2015-12-21,34.185788,-118.363082,1108321723,RED ZONE
3,363.0,2015-12-19,34.157607,-118.438092,1109139006,HANDICAP/NO DP ID
4,68.0,2015-12-26,34.126932,-118.311225,1109455266,PK IN PROH AREA


In [88]:
# convert "issue_date" column to a date 
new_df['issue_date'] = pd.to_datetime(new_df['issue_date'], dayfirst=True)

In [89]:
# sort dataframe rows between two dates-----(2014 - 2019)
start_date = "2014-01-01"
end_date = "2019-12-31"

mask = (new_df['issue_date'] > start_date) & (new_df['issue_date'] <= end_date)
new_df.loc[mask]
revenue_2014_2019 = new_df.loc[mask]

print(f'There are: {len(revenue_2014_2019)} records as of {end_date}')
print("=============================================================")
print(f'{len(new_df)-len(revenue_2014_2019)} records were disregarded')
revenue_2014_2019.head()

There are: 8076220 records as of 2019-12-31
109 records were disregarded


,fine_amount,issue_date,latitude,longitude,ticket_number,violation_description
0,58.0,2015-12-21,33.945212,-118.401151,1104803000,WHITE CURB
1,NaN,2015-12-26,33.945212,-118.401009,1104820732,17104h
2,93.0,2015-12-21,34.185788,-118.363082,1108321723,RED ZONE
3,363.0,2015-12-19,34.157607,-118.438092,1109139006,HANDICAP/NO DP ID
4,68.0,2015-12-26,34.126932,-118.311225,1109455266,PK IN PROH AREA


In [90]:
# preview the missing values and the % of missing values in each column
df = revenue_2014_2019
def missing_values_table(df):
        mis_val = df.isnull().sum()
        mis_val_percent = 100 * df.isnull().sum() / len(df)
        mis_val_table = pd.concat([mis_val, mis_val_percent], axis=1)
        mis_val_table_ren_columns = mis_val_table.rename(
        columns = {0 : 'Missing Values', 1 : '% of Total Values'})
        mis_val_table_ren_columns = mis_val_table_ren_columns[
            mis_val_table_ren_columns.iloc[:,1] != 0].sort_values(
        '% of Total Values', ascending=False).round(1)
        print ("Selected dataframe has " + str(df.shape[1]) + " columns.\n"      
            "There are " + str(mis_val_table_ren_columns.shape[0]) +
              " columns that have missing values.")
        return mis_val_table_ren_columns

missing_values_table(df)

Selected dataframe has 6 columns.
There are 4 columns that have missing values.


,Missing Values,% of Total Values
fine_amount,3024,0.0
violation_description,390,0.0
latitude,2,0.0
longitude,2,0.0


In [91]:
revenue_2014_2019_V2 = revenue_2014_2019.dropna(axis = 0, how ='any')

In [92]:
print("Old data frame length:", len(revenue_2014_2019), "\nNew data frame length:",  
       len(revenue_2014_2019_V2), "\nNumber of rows with at least 1 NA value: ", 
       (len(revenue_2014_2019)-len(revenue_2014_2019_V2))) 

Old data frame length: 8076220 
New data frame length: 8073177 
Number of rows with at least 1 NA value:  3043


In [93]:
# put year, month, and a day into separate columns
revenue_2014_2019_V2['Year'] = revenue_2014_2019_V2['issue_date'].dt.year
revenue_2014_2019_V2['Month'] = revenue_2014_2019_V2['issue_date'].dt.month
# citations_2015_2018['Day'] = citations_2015_2018['issue_date'].dt.day
revenue_2014_2019_V2.drop(["issue_date"], axis = 1, inplace = True)

revenue_2014_2019_V2.head()

/Users/pavelaltukhov/anaconda3/envs/meters_ml_3.6/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/Users/pavelaltukhov/anaconda3/envs/meters_ml_3.6/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/Users/pavelaltukhov/anaconda3/envs/meters_ml_3.6/lib/python3.6/site-packages/pandas/core/frame.py:3940: SettingWithCopyWarning: 
A value is trying to be set on a copy

,fine_amount,latitude,longitude,ticket_number,violation_description,Year,Month
0,58.0,33.945212,-118.401151,1104803000,WHITE CURB,2015,12
2,93.0,34.185788,-118.363082,1108321723,RED ZONE,2015,12
3,363.0,34.157607,-118.438092,1109139006,HANDICAP/NO DP ID,2015,12
4,68.0,34.126932,-118.311225,1109455266,PK IN PROH AREA,2015,12
5,68.0,34.126932,-118.311225,1109455270,PK IN PROH AREA,2015,12


In [114]:
# count citations by year and month
df_top_frequency = revenue_2014_2019_V2.groupby(['Year', 'fine_amount'])['ticket_number'].agg(
    {"counts": len}).sort_values("counts", ascending=False).reset_index()

df_top_frequency

/Users/pavelaltukhov/anaconda3/envs/meters_ml_3.6/lib/python3.6/site-packages/ipykernel_launcher.py:3: FutureWarning: using a dict on a Series for aggregation
is deprecated and will be removed in a future version
  This is separate from the ipykernel package so we can avoid doing imports until


,Year,fine_amount,counts
0,2017,73.0,725880
1,2018,73.0,610983
2,2016,73.0,603101
3,2015,73.0,426141
4,2015,63.0,426022
5,2016,63.0,411642
6,2017,63.0,405547
7,2018,63.0,316900
8,2017,68.0,314388
9,2018,68.0,265692


In [98]:
df_top_frequency['revenue'] = df_top_frequency.fine_amount * df_top_frequency.counts
df_top_frequency.head()

,Year,fine_amount,counts,revenue
0,2017,73.0,725880,52989240.0
1,2018,73.0,610983,44601759.0
2,2016,73.0,603101,44026373.0
3,2015,73.0,426141,31108293.0
4,2015,63.0,426022,26839386.0


In [99]:
df_top_frequency.drop(["fine_amount", "counts"], axis = 1, inplace = True)

In [111]:
df_top_frequency_2 = (df_top_frequency.groupby('Year')['revenue'].sum())
x = pd.DataFrame(df_top_frequency_2).reset_index()
x

,Year,revenue
0,2014,2371300.0
1,2015,107172665.0
2,2016,130124635.0
3,2017,151606561.0
4,2018,124737643.0
5,2019,49971119.0


In [110]:
x.to_csv("visualization_data/graph_3_revenue_by_yyyy.csv", encoding = 'utf-8', index = False)

# Map Visualization 2 | Areas with Highest Amount of Revenue

In [140]:
# loop over csv files and extract columns needed

path = '../data_collection_munging/parking_citations/data/citations_db_ready/'
all_files = glob.glob(path + "/*.csv")

li = []

for filename in all_files:
    df = pd.read_csv(filename, sep=",", usecols=[3, 5, 6, 7, 11])
    li.append(df)
new_df = pd.concat(li)
print(f'There are: {len(new_df)} records in dataframe')
new_df.head()

There are: 8076329 records in dataframe


,fine_amount,latitude,location,longitude,ticket_number
0,58.0,33.945212,200 WORLD WAY,-118.401151,1104803000
1,NaN,33.945212,100 WORLD WAY,-118.401009,1104820732
2,93.0,34.185788,10635 FRIAR ST,-118.363082,1108321723
3,363.0,34.157607,14006 RIVERSIDE DR,-118.438092,1109139006
4,68.0,34.126932,3200 CANYON DR,-118.311225,1109455266


In [141]:
map_2 = new_df.dropna(axis = 0, how ='any')

In [142]:
print("Old data frame length:", len(new_df), "\nNew data frame length:",  
       len(map_2), "\nNumber of rows with at least 1 NA value: ", 
       (len(new_df)-len(map_2))) 

Old data frame length: 8076329 
New data frame length: 8073293 
Number of rows with at least 1 NA value:  3036


In [160]:
# count citations by year and month
df_top_frequency = map_2.groupby(['latitude', 'longitude', 'location', 'fine_amount'])['ticket_number'].agg(
    {"counts": len}).sort_values("counts", ascending=False).reset_index()

df_top_frequency.head()

/Users/pavelaltukhov/anaconda3/envs/meters_ml_3.6/lib/python3.6/site-packages/ipykernel_launcher.py:3: FutureWarning: using a dict on a Series for aggregation
is deprecated and will be removed in a future version
  This is separate from the ipykernel package so we can avoid doing imports until


,latitude,longitude,location,fine_amount,counts
0,33.991691,-118.467910,1301 ELECTRIC AVE,63.0,6736
1,33.990358,-118.463691,1600 IRVING TABOR CT,63.0,5225
2,34.073023,-118.323530,101 LARCHMONT BL N,63.0,4230
3,34.038852,-118.443275,2110 CORINTH AVE,68.0,3656
4,34.147865,-118.378977,4301 TUJUNGA AV,63.0,3207


In [161]:
df_top_frequency['revenue'] = df_top_frequency.fine_amount * df_top_frequency.counts
df_top_frequency.head()

,latitude,longitude,location,fine_amount,counts,revenue
0,33.991691,-118.467910,1301 ELECTRIC AVE,63.0,6736,424368.0
1,33.990358,-118.463691,1600 IRVING TABOR CT,63.0,5225,329175.0
2,34.073023,-118.323530,101 LARCHMONT BL N,63.0,4230,266490.0
3,34.038852,-118.443275,2110 CORINTH AVE,68.0,3656,248608.0
4,34.147865,-118.378977,4301 TUJUNGA AV,63.0,3207,202041.0


In [162]:
df_top_frequency_1 = df_top_frequency[df_top_frequency.revenue > 2000]
df_top_frequency_1.head()

,latitude,longitude,location,fine_amount,counts,revenue
0,33.991691,-118.467910,1301 ELECTRIC AVE,63.0,6736,424368.0
1,33.990358,-118.463691,1600 IRVING TABOR CT,63.0,5225,329175.0
2,34.073023,-118.323530,101 LARCHMONT BL N,63.0,4230,266490.0
3,34.038852,-118.443275,2110 CORINTH AVE,68.0,3656,248608.0
4,34.147865,-118.378977,4301 TUJUNGA AV,63.0,3207,202041.0


In [163]:
df_top_frequency_1.drop(["fine_amount", "counts"], axis = 1, inplace = True)

/Users/pavelaltukhov/anaconda3/envs/meters_ml_3.6/lib/python3.6/site-packages/pandas/core/frame.py:3940: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


In [165]:
df_top_frequency_1.to_csv("visualization_data/map_2_revenue_areas.csv", encoding = 'utf-8', index = False)

# Graph 5 | Fines Distribution

In [122]:
# loop over csv files and extract columns needed

path = '../data_collection_munging/parking_citations/data/citations_db_ready/'
all_files = glob.glob(path + "/*.csv")

li = []

for filename in all_files:
    df = pd.read_csv(filename, sep=",", usecols=[3])
    li.append(df)
new_df = pd.concat(li)
print(f'There are: {len(new_df)} records in dataframe')
new_df.head()

There are: 8076329 records in dataframe


,fine_amount
0,58.0
1,NaN
2,93.0
3,363.0
4,68.0


In [123]:
# preview the missing values and the % of missing values in each column
df = new_df
def missing_values_table(df):
        mis_val = df.isnull().sum()
        mis_val_percent = 100 * df.isnull().sum() / len(df)
        mis_val_table = pd.concat([mis_val, mis_val_percent], axis=1)
        mis_val_table_ren_columns = mis_val_table.rename(
        columns = {0 : 'Missing Values', 1 : '% of Total Values'})
        mis_val_table_ren_columns = mis_val_table_ren_columns[
            mis_val_table_ren_columns.iloc[:,1] != 0].sort_values(
        '% of Total Values', ascending=False).round(1)
        print ("Selected dataframe has " + str(df.shape[1]) + " columns.\n"      
            "There are " + str(mis_val_table_ren_columns.shape[0]) +
              " columns that have missing values.")
        return mis_val_table_ren_columns

missing_values_table(df)

Selected dataframe has 1 columns.
There are 1 columns that have missing values.


,Missing Values,% of Total Values
fine_amount,3032,0.0


In [124]:
new_df_2 = new_df.dropna(axis = 0, how ='any')

In [125]:
new_df_2.to_csv("visualization_data/graph_4_fines_distribution.csv", encoding = 'utf-8', index = False)

# Graph 4 | Top 10 Revenue by Violation Type

In [80]:
# loop over csv files and extract columns needed

path = '../data_collection_munging/parking_citations/data/citations_db_ready/'
all_files = glob.glob(path + "/*.csv")

li = []

for filename in all_files:
    df = pd.read_csv(filename, sep=",", usecols=[3, 4, 5, 7, 11, 13])
    li.append(df)
new_df = pd.concat(li)
print(f'There are: {len(new_df)} records in dataframe')
new_df.head()

There are: 8076329 records in dataframe


,fine_amount,issue_date,latitude,longitude,ticket_number,violation_description
0,58.0,2015-12-21,33.945212,-118.401151,1104803000,WHITE CURB
1,NaN,2015-12-26,33.945212,-118.401009,1104820732,17104h
2,93.0,2015-12-21,34.185788,-118.363082,1108321723,RED ZONE
3,363.0,2015-12-19,34.157607,-118.438092,1109139006,HANDICAP/NO DP ID
4,68.0,2015-12-26,34.126932,-118.311225,1109455266,PK IN PROH AREA


In [81]:
# preview the missing values and the % of missing values in each column
df = new_df
def missing_values_table(df):
        mis_val = df.isnull().sum()
        mis_val_percent = 100 * df.isnull().sum() / len(df)
        mis_val_table = pd.concat([mis_val, mis_val_percent], axis=1)
        mis_val_table_ren_columns = mis_val_table.rename(
        columns = {0 : 'Missing Values', 1 : '% of Total Values'})
        mis_val_table_ren_columns = mis_val_table_ren_columns[
            mis_val_table_ren_columns.iloc[:,1] != 0].sort_values(
        '% of Total Values', ascending=False).round(1)
        print ("Selected dataframe has " + str(df.shape[1]) + " columns.\n"      
            "There are " + str(mis_val_table_ren_columns.shape[0]) +
              " columns that have missing values.")
        return mis_val_table_ren_columns

missing_values_table(df)

Selected dataframe has 6 columns.
There are 5 columns that have missing values.


,Missing Values,% of Total Values
fine_amount,3032,0.0
violation_description,390,0.0
latitude,4,0.0
longitude,4,0.0
issue_date,2,0.0


In [82]:
graph_5 = new_df.dropna(axis = 0, how ='any')

In [83]:
# count citations by year and month
df_top_frequency = graph_5.groupby(['violation_description', 'fine_amount'])['ticket_number'].agg(
    {"counts": len}).sort_values("counts", ascending=False).reset_index()

df_top_frequency

/Users/pavelaltukhov/anaconda3/envs/meters_ml_3.6/lib/python3.6/site-packages/ipykernel_launcher.py:3: FutureWarning: using a dict on a Series for aggregation
is deprecated and will be removed in a future version
  This is separate from the ipykernel package so we can avoid doing imports until


,violation_description,fine_amount,counts
0,NO PARK/STREET CLEAN,73.0,2253930
1,METER EXP.,63.0,1567269
2,PREFERENTIAL PARKING,68.0,605718
3,RED ZONE,93.0,598868
4,DISPLAY OF TABS,25.0,469563
5,NO PARKING,73.0,364415
6,DISPLAY OF PLATES,25.0,232441
7,WHITE ZONE,58.0,193299
8,PARKED OVER TIME LIMIT,58.0,167679
9,NO STOP/STANDING,93.0,149860


In [84]:
df_top_frequency['revenue'] = df_top_frequency.fine_amount * df_top_frequency.counts
df_top_frequency.head()

,violation_description,fine_amount,counts,revenue
0,NO PARK/STREET CLEAN,73.0,2253930,164536890.0
1,METER EXP.,63.0,1567269,98737947.0
2,PREFERENTIAL PARKING,68.0,605718,41188824.0
3,RED ZONE,93.0,598868,55694724.0
4,DISPLAY OF TABS,25.0,469563,11739075.0


In [85]:
df_top_frequency.to_csv("visualization_data/graph_5_revenue_by_violation.csv", encoding = 'utf-8', index = False)